# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"linked_variables":{}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,23.170741,0.500962,1234,23.671704,22.669779,47.343407,45.339557
6,612.591196,-0.405803,1234,612.185393,612.996999,1224.370786,1225.993998
7,459.052767,-0.059564,1234,458.993203,459.112332,917.986406,918.224663
8,210.799669,0.172631,1234,210.972300,210.627039,421.944600,421.254078
9,159.539567,0.483249,1234,160.022816,159.056318,320.045632,318.112637
10,125.055595,0.803667,1234,125.859261,124.251928,251.718523,248.503856
11,20.314013,-0.193967,1234,20.120046,20.507981,40.240092,41.015962
12,297.744486,-0.916609,1234,296.827877,298.661094,593.655755,597.322189
13,755.584923,0.561458,1234,756.146381,755.023465,1512.292762,1510.046929
14,102.562139,0.181797,1234,102.743936,102.380342,205.487873,204.760684


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-23.671704,22.669779
6,-612.185393,612.996999
7,-458.993203,459.112332
8,-210.972300,210.627039
9,-160.022816,159.056318
10,-125.859261,124.251928
11,-20.120046,20.507981
12,-296.827877,298.661094
13,-756.146381,755.023465
14,-102.743936,102.380342


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-23.671704,22.669779
6,-612.185393,612.996999
7,-458.993203,459.112332
8,-210.972300,210.627039
9,-160.022816,159.056318
10,-125.859261,124.251928
11,-20.120046,20.507981
12,-296.827877,298.661094
13,-756.146381,755.023465
14,-102.743936,102.380342


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[ -23.67170361,   22.66977869],
       [-612.18539277,  612.996999  ],
       [-458.99320275,  459.11233168],
       [-210.97229999,  210.62703898],
       [-160.02281618,  159.05631848],
       [-125.85926125,  124.25192818],
       [ -20.12004584,   20.50798081],
       [-296.82787737,  298.66109447],
       [-756.14638106,  755.02346466],
       [-102.74393643,  102.38034211]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,45.343407,-45.339557
6,1222.370786,-1225.993998
7,915.986406,-918.224663
8,419.944600,-421.254078
9,318.045632,-318.112637
10,249.718523,-248.503856
11,38.240092,-41.015962
12,591.655755,-597.322189
13,1510.292762,-1510.046929
14,203.487873,-204.760684


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
